# Getting started with UCC state preparation in VQE

The package ``qat.fermion.chemistry.ucc`` prodives all the necessary elements to study a molecule with the *variational quantum eigensolver* (VQE) using *unitary coupled cluster* (UCC) state preparation. 

As a reminder, the **VQE** solves the following minimization problem :
$$
E = \min_{\vec{\theta}}\; \langle \psi(\vec{\theta}) \,|\, \hat{H} \,|\, \psi(\vec{\theta}) \rangle
$$
where $\hat{H}$ is the molecular Hamiltonian (at fixed geometry) and $|\psi(\vec{\theta})\rangle$ is a parametrized state used to explore the state space.

An effective state preparation method is given by **UCC**. The state is then:
$$
|\psi(\vec{\theta})\rangle = e^{\hat{T}(\vec{\theta}) - \hat{T}^\dagger(\vec{\theta})} |0\rangle
$$
where $\hat{T}(\theta)$ is the *cluster operator*. Its non-truncated form is given by:
$$
\hat{T}(\vec{\theta}) = \hat{T}_1(\vec{\theta}) + \hat{T}_2(\vec{\theta}) + \cdots
$$
where
$$
\hat{T}_1 = \sum_{a\in U}\sum_{i \in O} \theta_a^i\, \hat{a}_a^\dagger \hat{a}_i \qquad
\hat{T}_2 = \sum_{a>b\in U}\sum_{i>j\in O} \theta_{a, b}^{i, j}\, \hat{a}^\dagger_a \hat{a}^\dagger_b \hat{a}_i \hat{a}_j \qquad
\cdots
$$
($O$ is the set of occupied orbitals and $U$, the set of unoccipied ones.)

This notebook will describe the basics of the ``ucc`` module by first introducing the one-liner and then, describing the execution flow, subfunction by subfunction.

## Direct execution

The most compact form of the procedure is given by ``uccsd``.

The first step is to instantiate OpenFermion's ``MolecularData`` class with the chosen basis set and the studied geometry.

Then this object is given as argument to the function which outputs:
- the parametric state preparation (as a function outputing a QRoutine),
- the initial parameter set,
- the Hamiltonian,
- the number of qubits.

These elements can then be used in a VQE algorithm.

*Note: Both ``qat.fermion.chemistry.ucc`` and the ``MolecularData`` from ``openfermion.hamiltonians`` need to be imported.*

**Example:** dihydrogen in STO-3G basis at 0.7414 Angström (internuclear distance.)

In [1]:
# Package import:
from qat.fermion.chemistry.deprecated.ucc import uccsd
from openfermion.hamiltonians import MolecularData

# Molecule construction:
basis = 'sto-3g'
multiplicity = 1 # Supposed to be set to 1 (i.e. singlet state is assumed!)
bond_length = 0.7414 # In Angstrom.
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]

molecule = MolecularData(geometry, basis, multiplicity)

In [2]:
qroutine_uccsd, theta_0, h_spin, nb_qubits = uccsd(molecule)

# Optional parameters are:
#  - The transformation: transformation='bk',
#  - The numerical threshold: threshold=1e-15, 
#  - The active space selection criterion: as_selection=0.001,
#        - either the lower limit value for the NOONs of active orbitals, 
#        - or the list of active orbitals.  
#  - The numbers of single and double excitations to consider: max_nb_single_ex=0, max_nb_double_ex=3, 
#  - The transformation as a BinaryCode, for user-defined ones: code=None, 
#  - The reduction of trivially acted-upon qubits (valid for Braavyi-Kitaev, in certain basis): reduction=False, 
#  - The approximation of double excitation excitations transformed by Bravyi-Kitaev: approx=True.

/home/tayral/.my_python/lib64/python3.6/site-packages/pyscf/scf/chkfile.py:31: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(chkfile) as fh5:
/home/tayral/.my_python/lib64/python3.6/site-packages/pyscf/lib/misc.py:876: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  h5py.File.__init__(self, filename, *args, **kwargs)


In [3]:
print(h_spin)

And now, the VQE execution:

In [4]:
# Definition of the QPU, e.g. LinAlg:
from qat.linalg import get_qpu_server as get_linalg_qpu
linalg_qpu = get_linalg_qpu()
# Definition of the optimizer, e.g. SPSA:
from qat.fermion.optimization import Optimizer
from qat.fermion.spsa import spsa_minimize
spsa_minimizer = Optimizer(spsa_minimize)

from qat.fermion.hamiltonian import IsingHamiltonian
h_vqe = IsingHamiltonian(list_pauli_operators=h_spin.list_pauli_operators,
                         list_pauli_values = h_spin.list_pauli_values)

# Execution of the VQE:
from qat.fermion.vqe import VQE
e, param, nb_evals, energies = VQE(h_vqe,  spsa_minimizer, qroutine_uccsd, theta_0, 
                                   linalg_qpu, grouping=False, display=True)

Precision reached ( 0.0001 ), iteration number = 8
Energy = (-1.1372701746002185+0j) Optimized parameters = [0.11306201+0.j] 
 Number of function evaluations = 8


In [5]:
print("VQE energy=", e.real)

VQE energy= -1.1372701746002185


## Step-by-step execution

We now break up the high-level function ``uccsd`` into its basic steps.

Package import:

In [6]:
# Package import:
from qat.fermion.chemistry.deprecated.tools import compute_noons, select_active_orbitals, build_molecular_description,\
                                restrict_to_active_space
from qat.fermion.chemistry.deprecated.ucc import init_uccsd, select_excitation_operators, build_cluster_operator, build_ucc_ansatz
from qat.fermion.chemistrystry.deprecated.remapping import transform_all, remap_all, reduce_qubit_number
from chemistryermion.chemistry.deprecated.util import from_QubitOperator_to_IsingHamiltonian
from openfermion.hamiltonians import MolecularData

ImportError: cannot import name 'transform_all'

### 0. Input

To instanciate the ``MolecularData`` class, one needs to select the basis set and the geometry of the molecule (i.e. atoms and nuclear positions.) The ``multiplicity`` parameter should be be set to 1, as ground states are studied (and they are singlet state.)

In [ ]:
# Molecule construction:
basis = 'sto-3g'
multiplicity = 1
bond_length = 0.7414 # In Angstrom.
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]

molecule = MolecularData(geometry, basis, multiplicity)

### 1. Active space selection
#### a. Reference data

This function computes the different data pieces needed for the state
preparation and the definition of the Hamiltonian.

The Restricted Hartree-Fock (RHF) and truncated Configuration
Interaction (CISSD) methods are executed on the molecule passed as
various necessary elements are outputed:
(spatial) orbital number, electron number, one- and two-electron
integrals, orbital energies, natural orbital occupation numbers...

In [ ]:
mol_hf, nb_e, noons, orb_energies = compute_noons(molecule) 

print("Electron number =", nb_e)
print("Natural orbital occupation numbers:", noons)

#### b. Active space

In order to reduce the size of the problem, an **active space** is defined.

By using the CISD comutation, one can get a hint of the occupancy of spatial orbitals. Active ones can then be  easily selected by the following procedure:
1. (Already done) Compute natural orbitals occupancy numbers (NOONs) with CISD or CCSD (NOONs are the eigenvalues of 1-body density matrix) ;
2. Categorize i-th orbital as: 
    - Inactive occupied (*core*) i.f.f. 
         * $n_i \geqslant 1.98$, and
         * It is not the last one occupied ( $\implies$ at least, one active occupied).
    - Inactive unoccupied i.f.f. $n_i \leqslant n_{\mathrm{crit}}$ .
3. Reduce the problem to active space (implies correction to Hamiltonian coefficients).

In [ ]:
active_spin_orbs, active_spatial_orbs, inactive_occupied_spatial_orbs = select_active_orbitals(noons, nb_e,
                                                                                               as_selection=1e-3)

#### c. Restriction to the active space

The relevant data elements are restricted to the selected active.

In [ ]:
nb_active_spin_orbs, nb_active_electrons, noons_active,\
    orb_energies_active, reindexed_active_orbs = restrict_to_active_space(active_spin_orbs, 
                                                                          inactive_occupied_spatial_orbs,
                                                                          nb_e,
                                                                          noons, 
                                                                          orb_energies)

### 2. Construction of the Hamiltonian

With the defined active space and the output of the HF method, the molecular Hamiltonian can be built as:
$$
\hat{H} = c \hat{I} +
\sum_{p, q} h_{p, q}\, \hat{a}^\dagger_p \hat{a}_q +
\frac{1}{2}\sum_{p,q,r,s} h_{p, q, r, s}\, \hat{a}^\dagger_p \hat{a}^\dagger_q \hat{a}_r \hat{a}_s
$$
where one- and two-electron coefficients, $h_{p, q}$ and $h_{p, q, r, s}$, are computed from HF by orbital transformation.

In [ ]:
hamiltonian_of, two_body_coefficients = build_molecular_description(mol_hf,
                                                                    inactive_occupied_spatial_orbs,
                                                                    active_spatial_orbs)

### 3. Cluster operator construction
#### a. Construction of the initial state and parametrization

This function builds the HF initial state by simply filling orbitals (fully) from inside to outside.

Also, as MP2 gives a better initial guess, the trial parametrization is built from the following equations (obtained by identification in MP2 and CC equation):
$$
\theta_a^i = 0 \qquad 
    \theta_{a, b}^{i, j} = \frac{h_{a, b, j, i} -
    h_{a, b, i, j}}{\epsilon_i + \epsilon_j -\epsilon_a -
    \epsilon_b}
$$
where $\epsilon_i$ is the energy of the $i$-th orbital.

In [ ]:
ket_hf_init, active_occupied_orbs, active_unoccupied_orbs, theta_init = init_uccsd(nb_active_spin_orbs, 
                                                                                   nb_active_electrons, 
                                                                                   two_body_coefficients, 
                                                                                   reindexed_active_orbs,
                                                                                   orb_energies_active)

#### b. Selection of the excitation operators

Based on the parameters limiting the number of one- and two-electron excitation terms, the cluster operator $T$ is built. 

Actually, it is an implementation of UCCSpD as in UCC "single - paired double" because only **single** and **singlet double** excitation terms are taken into account, i.e. of the form
$$
\hat{a}^\dagger_a \hat{a}_i \qquad \text{or} \qquad
\hat{a}^\dagger_{k, \uparrow}\hat{a}^\dagger_{k, \downarrow} \hat{a}_{l, \uparrow}\hat{a}_{l, \downarrow}
$$
where $k, l$ are *spatial* indices.

In [ ]:
l_ex_op = select_excitation_operators(noons_active, 
                                      active_occupied_orbs, 
                                      active_unoccupied_orbs, 
                                      max_nb_single_ex=0,
                                      max_nb_double_ex=3)     

####  c. Construction of the cluster operator

Based on the selected excitation operators, the cluster operator is described as a dictionary within OpenFermion's formalism (the keys are the normal-ordered excitation indices and the values are FermionOperator instances.)

Moreover, $\vec{\theta}$ is also mapped as a dictionary.

In [ ]:
t_opti, theta_bis = build_cluster_operator(l_ex_op, theta_init)

### 4. Mapping to qubit (spin) space

####  a. Fermion-qubit transformation

The selected transformation is applied to the operators **and** to the HF state. Indeed, some transforms (such as Bravyi-Kitaev) change the encoding of the orbitals and thus, call for some modification of the initial state;

Here, the transformations are used as implemented by OpenFermion (be it directly or through user-defined BinaryCode instances.)

In [ ]:
h_transformed_of, t_transformed, ket_hf_transformed = transform_all(hamiltonian_of, 
                                                                 t_opti, "bk", ket_hf_init, len(active_spin_orbs))

#### b. Remapping

Unoccupied qubits that do not experience a population change can be eliminated as they do not contribute.

``qubit_mapping`` produces the new numbering of qubits and ``remap_all`` applies it to the relevant elements.

**Beware** high numerical sensivity have been experienced with classical eigensolvers. Size reduction helps improve outputs.

In [ ]:
qubit_mapping = reduce_qubit_number(h_transformed_of, ket_hf_transformed)

t_reduced, ket_hf_reduced, theta_0, h_reduced_of = remap_all(t_transformed, 
                                                             ket_hf_transformed, 
                                                             theta_bis, 
                                                             h_transformed_of, 
                                                             qubit_mapping)

### 5. UCC preparation circuit

The parametric QRoutine is output by the ``build_ucc_ansatz`` function. It implements the unitary operator $\hat{U}$ such that:
$$
\begin{align*}
Q \vert \vec{0} \rangle   
    &= \vert \mathrm{UCC} (\vec{\theta}) \rangle \\
    &= e^{T(\vec{\theta})} \vert \mathrm{HF} \rangle   
\end{align*}
$$

In [ ]:
qroutine_uccsd = build_ucc_ansatz(t_reduced, ket_hf_reduced)

### 6. Conversion to IsingHamiltonian

And finally, the Hamiltonian is described in the right formalism:

In [ ]:
h_spin = from_QubitOperator_to_IsingHamiltonian(h_reduced_of, len(ket_hf_reduced))
h_vqe = IsingHamiltonian(list_pauli_operators=h_spin.list_pauli_operators,
                         list_pauli_values=h_spin.list_pauli_values)

### 7. VQE Execution

As with the direct execution:

In [ ]:
# Definition of the optimizer, e.g. SPSA:
from qat.fermion.optimization import Optimizer
from qat.fermion.spsa import spsa_minimize
spsa_minimizer = Optimizer(spsa_minimize)

# Execution of the VQE:
from qat.fermion.vqe import VQE
e, param, nb_evals, energies = VQE(h_vqe, spsa_minimizer, qroutine_uccsd, theta_0, 
                                   linalg_qpu, grouping=False, display=True)

In [ ]:
e.real